# Train multiple linear probes at once
In this notebook we will find out at which layer a transformer has the most linearly seperable information required to do causal language modelling on wikitext data. The nice thing about *transformer_heads* is that this will all be possible with just one training run.

In [1]:
from transformer_heads import load_headed
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import evaluate_head_wise, get_top_n_preds
import torch

In [ ]:
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100

In [ ]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

Let's define a lot of heads in a loop. The heads will be hooked at layer -1,-3,-5,-7,-9,-11,-13. This is using python indexing: Layer -1 means after the last transformer block for example. We'll keep the original pretrained lm_head of the transformer model for comparison.

In [3]:
head_configs = [
    HeadConfig(
        name=f"wikitext_head_{(1+(i-1)*2)}",
        layer_hook=-(1 + (i - 1) * 2),
        in_size=hidden_size,
        hidden_size=0,
        num_layers=1,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
    )
    for i in range(1, 8)
]
head_configs.append(
    HeadConfig(
        name=f"lm_head",
        layer_hook=-1,
        in_size=hidden_size,
        hidden_size=0,
        num_layers=1,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
        trainable=False,
    )
)

In [4]:
dd = load_dataset("wikitext", "wikitext-2-v1")

In the *tokenize_function*, we define labels for each head. For causal_lm, this is just the copied input_ids.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    for hc in head_configs:
        out[hc.name] = out["input_ids"].copy()
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].map(tokenize_function, batched=True)
dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns("text")

Map:   0%|          | 0/23627 [00:00<?, ? examples/s]

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = load_headed(
    model_class,
    model_path,
    head_configs=head_configs,
    quantization_config=quantization_config,
    device_map={"": torch.cuda.current_device()},
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.wikitext_head_1.lins.0.weight', 'heads.wikitext_head_11.lins.0.weight', 'heads.wikitext_head_13.lins.0.weight', 'heads.wikitext_head_3.lins.0.weight', 'heads.wikitext_head_5.lins.0.weight', 'heads.wikitext_head_7.lins.0.weight', 'heads.wikitext_head_9.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Huggingface tells us that our newly added heads are newly initialized. Great.

In [7]:
print_trainable_parameters(model)

all params: 352154112 || trainable params: 270181632 || trainable%: 76.72255492504372
params by dtype: defaultdict(<class 'int'>, {torch.float32: 309686784, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 270181632})


A lot of heads with large vocab size -> High amount of trainable parameters

In [8]:
dd["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'wikitext_head_1', 'wikitext_head_3', 'wikitext_head_5', 'wikitext_head_7', 'wikitext_head_9', 'wikitext_head_11', 'wikitext_head_13', 'lm_head'],
    num_rows: 23627
})

In [9]:
print(get_top_n_preds(5, model, "The historical significance of", tokenizer))

{'wikitext_head_1': [' unlocking', ' dancer', 'slot', ' Dise', 'Find'], 'wikitext_head_3': ['309', 'Students', 'AIN', 'II', 'mn'], 'wikitext_head_5': [' puzzle', ' cites', ' Courtesy', ' Crisis', ' courses'], 'wikitext_head_7': ['UST', ' admire', ' tubes', ' assassin', ' Ely'], 'wikitext_head_9': [' Mom', ' NG', 'School', 'icts', ' technically'], 'wikitext_head_11': ['Yang', "')", ' )))', ' regional', ' dreams'], 'wikitext_head_13': ['654', 'Email', ' costly', ' proc', ' admirable'], 'lm_head': [' the', ' this', ' these', ' that', ' his']}


The untrained heads are predicting somewhat randomly

In the collator, we need to make sure that the labels for each head are padded correctly. Here, we are padding with -100, the ignore_index token for cross_entropy.

In [10]:
args = TrainingArguments(
    output_dir="linear_probe_test",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
)
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
        **{key.name: -100 for key in head_configs},
    }
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
20,102.691700
40,88.865100
60,79.065600
80,71.484000
100,67.034600
120,62.662200
140,59.936500
160,57.143900
180,55.878300
200,54.893500


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/u/ykeller/conda-envs/sh_finetuning/lib/

TrainOutput(global_step=2954, training_loss=43.94008915926366, metrics={'train_runtime': 1227.1579, 'train_samples_per_second': 19.253, 'train_steps_per_second': 2.407, 'total_flos': 1.2795692896977408e+16, 'train_loss': 43.94008915926366, 'epoch': 1.0})

In [11]:
print(evaluate_head_wise(model, dd["validation"], collator, epochs=eval_epochs))

Evaluating:   0%|          | 0/308 [00:00<?, ?it/s]/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Evaluating: 100%|██████████| 308/308 [04:30<00:00,  1.14it/s]

(37.688800923236, {'wikitext_head_1': 4.534386265587497, 'wikitext_head_3': 4.244516681541096, 'wikitext_head_5': 4.476744795774485, 'wikitext_head_7': 4.685033707649676, 'wikitext_head_9': 4.8696470934075196, 'wikitext_head_11': 4.835106800902974, 'wikitext_head_13': 5.5660051819566005, 'lm_head': 4.4773603600341})


That one is some interesting result. Despite the fact that gpt2 was pretrained to do causal_lm after the last transformer block, the linear probe hooked after the third to last block performs the best for causal_lm on wikitext data.

In [15]:
print(get_top_n_preds(5, model, "The historical significance of", tokenizer))

{'wikitext_head_1': [' the', ' his', ' this', ' a', ' its'], 'wikitext_head_3': [' the', ' this', ' his', ' these', ' <'], 'wikitext_head_5': [' the', ' this', ' these', ' its', ' his'], 'wikitext_head_7': [' the', ' this', ' his', ' <', ' these'], 'wikitext_head_9': [' the', ' this', ' <', ' a', ' his'], 'wikitext_head_11': [' the', ' a', ' <', ' his', ' this'], 'wikitext_head_13': [' the', ' a', ' <', ' his', '@'], 'lm_head': [' the', ' this', ' these', ' a', ' his']}


The heads are now predicting pretty likely tokens. Note that predicting *<* is an artefact of the wikitext data.